In [1]:
%load_ext autoreload
%autoreload 2
%matplotlib inline
%config InlineBackend.figure_format = 'svg'

In [2]:
cd /Users/fred/Umila/numila

/Users/fred/Umila/numila


In [3]:
from sklearn import metrics
import pandas as pd
import itertools
import matplotlib.pyplot as plt
import seaborn
seaborn.set_style('whitegrid')
seaborn.set_palette('muted')

from main import *

In [4]:
# langs = ['English', 'Farsi', 'German', 'Japanese', 'Spanish']
# langs = ['toy2']
langs = ['English',] # 'Spanish', 'German']
langs = [setup(lang, 4000) for lang in langs]

TypeError: unsupported operand type(s) for ** or pow(): 'HoloNode' and 'float'

# ROC

In [ ]:
rocs = itertools.chain.from_iterable(roc_lang(lang) for lang in langs)
roc_df = pd.DataFrame(list(rocs))

In [ ]:
def roc_plot(df):
    ax = plt.subplot(1,1,1)
    plt.plot([0, 1], [0, 1], 'k--')
    plt.xlim([0.0, 1.0])
    plt.ylim([0.0, 1.05])
    plt.xlabel('False Positive Rate')
    plt.ylabel('True Positive Rate')

    for i, row in df.iterrows():
        plt.plot(row.fpr, row.tpr, label='{0} (area = {1:0.2f})'
                 .format(row.model, row['ROC area under curve']))

    # Sort labels in legend.
    handles, labels = ax.get_legend_handles_labels()
    def get_aoc(handle_label):
        label = handle_label[1]
        return float(label[-5:-1])
    hl = sorted(zip(handles, labels), key=get_aoc, reverse=True)
    handles2, labels2 = zip(*hl)
    ax.legend(handles2, labels2, loc=4, frameon=True)

    plt.gca().set_aspect('equal', adjustable='box')
    plt.gcf().set_size_inches(4, 4)

In [ ]:
# Plot roc curve for English.
# roc_plot(roc_df.query("language == 'English'"))
roc_plot(roc_df)
plt.savefig('figs/roc.pdf')

In [ ]:
# Plot area under curve for all languages.
seaborn.factorplot('model', 'ROC area under curve', hue='language', data=roc_df, kind='bar')
plt.savefig('all_roc.pdf')

# BLEU

In [ ]:
bleu_df = pd.concat(production.bleu_sim(lang) for lang in langs)

In [ ]:
seaborn.factorplot(x='model', hue='language', y='BLEU', data=bleu_df, kind='bar',
#             order=['holo', 'compose', 'prob',  'trigram']
               )
# plt.legend().draw_frame(True)
plt.savefig('figs/bleu.pdf')

In [ ]:
# full_df = roc_df[['ROC area under curve', 'model', 'language']].rename(columns={'ROC area under curve': 'measure'}).head()

full_df = pd.concat([pd.melt(roc_df, id_vars=['model', 'language'],
                   value_vars=['ROC area under curve'], var_name='measure'), 
           pd.melt(bleu_df, id_vars=['model', 'language'],
                   value_vars=['BLEU'], var_name='measure')])
    
seaborn.factorplot('model', 'value', hue='language', col='measure', data=full_df,
                   kind='bar', col_order=['BLEU', 'ROC area under curve'],
                   order=['holo', 'prob', 'trigram', 'random'])
plt.savefig('figs/combined.pdf')